In [ ]:
# import os
# import zipfile

# def zip_directory(folder_path, output_zip_path):
#     with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
#         for root, dirs, files in os.walk(folder_path):
#             for file in files:
#                 file_path = os.path.join(root, file)
#                 arcname = os.path.relpath(file_path, folder_path)
#                 zipf.write(file_path, arcname)

# # Example usage
# folder_to_zip = '../home'  # Replace with your folder path
# output_zip_file = '../machine_translation.zip'  # Replace with your desired output zip file name

# zip_directory(folder_to_zip, output_zip_file)

In [1]:
!python -V

Python 3.11.0rc1


In [2]:
!pip freeze

absl-py==2.0.0
anyio==4.3.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arrow==1.3.0
asttokens==2.4.1
astunparse==1.6.3
attrs==23.2.0
beautifulsoup4==4.12.3
bleach==6.1.0
blinker==1.4
cachetools==5.3.1
certifi==2023.7.22
cffi==1.16.0
charset-normalizer==3.2.0
click==8.1.7
cloudpickle==3.0.0
comm==0.2.2
contourpy==1.2.1
cryptography==3.4.8
cycler==0.12.1
dbus-python==1.2.18
debugpy==1.8.1
decorator==5.1.1
defusedxml==0.7.1
distro==1.7.0
dm-tree==0.1.8
entrypoints==0.4
et-xmlfile==1.1.0
etils==1.8.0
executing==2.0.1
fastjsonschema==2.19.1
flatbuffers==23.5.26
fonttools==4.51.0
fqdn==1.5.1
fsspec==2024.3.1
gast==0.5.4
google-auth==2.23.1
google-auth-oauthlib==1.0.0
google-pasta==0.2.0
googleapis-common-protos==1.63.0
grpcio==1.58.0
gym==0.26.2
gym-notices==0.0.8
h5py==3.9.0
httplib2==0.20.2
idna==3.4
importlib-metadata==4.6.4
importlib_resources==6.4.0
ipykernel==6.29.4
ipython==8.24.0
ipython-genutils==0.2.0
isoduration==20.11.0
jedi==0.19.1
jeepney==0.7.1
Jinja2

# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [3]:
%load_ext autoreload
%aimport helper, project_tests
%autoreload 1

2024-07-30 18:35:14.114045: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 18:35:14.114104: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 18:35:14.114157: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 18:35:14.124901: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Should return keras==2.14.0 and tf-keras==2.14.1
!pip freeze | grep keras

keras==2.14.0
tf-keras==2.14.1


In [5]:
# Should return tensorflow==2.14.0
!pip freeze | grep tensorflow

tensorflow==2.14.0
tensorflow-datasets==4.9.4
tensorflow-estimator==2.14.0
tensorflow-hub==0.16.1
tensorflow-io-gcs-filesystem==0.34.0
tensorflow-metadata==1.15.0


In [6]:
import collections

import helper
import numpy as np
import project_tests as tests

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [7]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [8]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [9]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [10]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)

    return tokenizer.texts_to_sequences(x), tokenizer
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [11]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length is None:
        length = max([len(seq) for seq in x])
    return pad_sequences(x, maxlen=length, padding='post')
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [12]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [13]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [14]:
BATCH_SIZE = 64
EPOCH = 10

### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [15]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Hyperparameters
    learning_rate = 0.001
    gru_units = 128
    activation = 'tanh'
    
    # Model architecture
    input_seq = Input(shape=input_shape[1:])
    rnn = GRU(gru_units, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(rnn)
    output = Activation(activation)(logits)

    model = Model(input_seq, output)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=BATCH_SIZE, epochs=EPOCH, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

2024-07-30 18:35:31.253483: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-30 18:35:31.265175: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-30 18:35:31.268524: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/10


2024-07-30 18:35:35.151954: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-07-30 18:35:38.842509: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x796d84070240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-30 18:35:38.842590: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-07-30 18:35:38.848520: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-30 18:35:39.008463: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1724/1724 [==============================] - 22s 9ms/step - loss: 1.5542 - accuracy: 0.6071 - val_loss: nan - val_accuracy: 0.6556
Epoch 2/10
1724/1724 [==============================] - 15s 8ms/step - loss: 1.1300 - accuracy: 0.6616 - val_loss: nan - val_accuracy: 0.6685
Epoch 3/10
1724/1724 [==============================] - 14s 8ms/step - loss: 1.0376 - accuracy: 0.6735 - val_loss: nan - val_accuracy: 0.6775
Epoch 4/10
1724/1724 [==============================] - 15s 8ms/step - loss: 0.9816 - accuracy: 0.6844 - val_loss: nan - val_accuracy: 0.6877
Epoch 5/10
1724/1724 [==============================] - 14s 8ms/step - loss: 0.9219 - accuracy: 0.7069 - val_loss: nan - val_accuracy: 0.7195
Epoch 6/10
1724/1724 [==============================] - 14s 8ms/step - loss: 0.8623 - accuracy: 0.7269 - val_loss: nan - val_accuracy: 0.7287
Epoch 7/10
1724/1724 [==============================] - 14s 8ms/step - loss: 0.8213 - accuracy: 0.7403 - val_loss: nan - val_accuracy: 0.7483
Epoch 8/10
1724/1

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [16]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    # Hyperparameters
    learning_rate = 0.001
    gru_units = 128
    embedding_dim = 128
    activation = 'tanh'

    # Model architecture
    input_seq = Input(shape=input_shape[1:])
    embedding = Embedding(english_vocab_size, embedding_dim)(input_seq)
    rnn = GRU(gru_units, return_sequences=True)(embedding)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(rnn)    
    output = Activation(activation)(logits)

    model = Model(input_seq, output)
    model.compile(loss=sparse_categorical_crossentropy,
                    optimizer=Adam(learning_rate),
                    metrics=['accuracy'])
    return model



tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=BATCH_SIZE, epochs=EPOCH, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
1724/1724 [==============================] - 25s 13ms/step - loss: 0.9897 - accuracy: 0.7616 - val_loss: nan - val_accuracy: 0.8719
Epoch 2/10
1724/1724 [==============================] - 18s 10ms/step - loss: 0.3425 - accuracy: 0.8925 - val_loss: nan - val_accuracy: 0.9047
Epoch 3/10
1724/1724 [==============================] - 18s 10ms/step - loss: 0.2702 - accuracy: 0.9122 - val_loss: nan - val_accuracy: 0.9166
Epoch 4/10
1724/1724 [==============================] - 18s 10ms/step - loss: 0.2395 - accuracy: 0.9207 - val_loss: nan - val_accuracy: 0.9236
Epoch 5/10
1724/1724 [==============================] - 18s 10ms/step - loss: 0.2212 - accuracy: 0.9256 - val_loss: nan - val_accuracy: 0.9267
Epoch 6/10
1724/1724 [==============================] - 18s 10ms/step - loss: 0.2088 - accuracy: 0.9286 - val_loss: nan - val_accuracy: 0.9277
Epoch 7/10
1724/1724 [==============================] - 18s 11ms/step - loss: 0.2021 - accuracy: 0.9303 - val_loss: nan - val_accuracy: 0.9282

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [17]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    # Hyperparameters
    learning_rate = 0.001
    gru_units = 128
    activation = 'tanh'

    # Model architecture
    input_seq = Input(shape=input_shape[1:])
    rnn = Bidirectional(GRU(gru_units, return_sequences=True))(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(rnn)
    output = Activation(activation)(logits)

    model = Model(input_seq, output)
    model.compile(loss=sparse_categorical_crossentropy,
                    optimizer=Adam(learning_rate),
                    metrics=['accuracy'])
    return model


tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1)) # Reshaping to work with bidirectional RNN

bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=BATCH_SIZE, epochs=EPOCH, validation_split=0.2)

print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
1724/1724 [==============================] - 30s 15ms/step - loss: 1.3117 - accuracy: 0.6451 - val_loss: nan - val_accuracy: 0.6854
Epoch 2/10
1724/1724 [==============================] - 24s 14ms/step - loss: 0.9612 - accuracy: 0.6963 - val_loss: nan - val_accuracy: 0.7067
Epoch 3/10
1724/1724 [==============================] - 24s 14ms/step - loss: 0.8706 - accuracy: 0.7095 - val_loss: nan - val_accuracy: 0.7157
Epoch 4/10
1724/1724 [==============================] - 24s 14ms/step - loss: 0.8182 - accuracy: 0.7189 - val_loss: nan - val_accuracy: 0.7244
Epoch 5/10
1724/1724 [==============================] - 24s 14ms/step - loss: 0.7770 - accuracy: 0.7283 - val_loss: nan - val_accuracy: 0.7357
Epoch 6/10
1724/1724 [==============================] - 24s 14ms/step - loss: 0.7438 - accuracy: 0.7383 - val_loss: nan - val_accuracy: 0.7395
Epoch 7/10
1724/1724 [==============================] - 25s 14ms/step - loss: 0.7083 - accuracy: 0.7522 - val_loss: nan - val_accuracy: 0.7602

### Model 4: Encoder-Decoder (IMPLEMENTATION)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [18]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    # Hyperparameters
    learning_rate = 0.001
    gru_units = 128
    activation = 'tanh'

    # Model architecture
    input_seq = Input(shape=input_shape[1:])
    encoder = GRU(gru_units)(input_seq)
    encoder = RepeatVector(output_sequence_length)(encoder)
    decoder = GRU(gru_units, return_sequences=True)(encoder)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder)
    output = Activation(activation)(logits)

    model = Model(input_seq, output)
    model.compile(loss=sparse_categorical_crossentropy,
                    optimizer=Adam(learning_rate),
                    metrics=['accuracy'])
    return model
tests.test_encdec_model(encdec_model)


# TODO: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1)) # Reshaping to work with encoder-decoder model

encdec_rnn_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=BATCH_SIZE, epochs=EPOCH, validation_split=0.2)

print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
1724/1724 [==============================] - 29s 14ms/step - loss: 1.9065 - accuracy: 0.5457 - val_loss: nan - val_accuracy: 0.6059
Epoch 2/10
1724/1724 [==============================] - 22s 13ms/step - loss: 1.3316 - accuracy: 0.6247 - val_loss: nan - val_accuracy: 0.6344
Epoch 3/10
1724/1724 [==============================] - 21s 12ms/step - loss: 1.2053 - accuracy: 0.6453 - val_loss: nan - val_accuracy: 0.6515
Epoch 4/10
1724/1724 [==============================] - 21s 12ms/step - loss: 1.1184 - accuracy: 0.6610 - val_loss: nan - val_accuracy: 0.6732
Epoch 5/10
1724/1724 [==============================] - 21s 12ms/step - loss: 1.0349 - accuracy: 0.6771 - val_loss: nan - val_accuracy: 0.6842
Epoch 6/10
1724/1724 [==============================] - 21s 12ms/step - loss: 0.9661 - accuracy: 0.6913 - val_loss: nan - val_accuracy: 0.6940
Epoch 7/10
1724/1724 [==============================] - 21s 12ms/step - loss: 0.9090 - accuracy: 0.7019 - val_loss: nan - val_accuracy: 0.7051

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [19]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    # Hyperparameters
    learning_rate = 0.001
    gru_units = 128
    embedding_dim = 128
    activation = 'tanh'

    # Model architecture
    input_seq = Input(shape=input_shape[1:])
    embedding = Embedding(english_vocab_size, embedding_dim)(input_seq)
    encoder = Bidirectional(GRU(gru_units))(embedding)
    encoder = RepeatVector(output_sequence_length)(encoder)
    decoder = Bidirectional(GRU(gru_units, return_sequences=True))(encoder)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder)
    output = Activation(activation)(logits)

    model = Model(input_seq, output)
    model.compile(loss=sparse_categorical_crossentropy,
                    optimizer=Adam(learning_rate),
                    metrics=['accuracy'])
    return model

tests.test_model_final(model_final)


print('Final Model Loaded')
# TODO: Train the final model
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

final_rnn_model = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

final_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=BATCH_SIZE, epochs=EPOCH, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(final_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Final Model Loaded
Epoch 1/10
1724/1724 [==============================] - 52s 25ms/step - loss: 1.2689 - accuracy: 0.6745 - val_loss: nan - val_accuracy: 0.7490
Epoch 2/10
1724/1724 [==============================] - 38s 22ms/step - loss: 0.6449 - accuracy: 0.8097 - val_loss: nan - val_accuracy: 0.8531
Epoch 3/10
1724/1724 [==============================] - 38s 22ms/step - loss: 0.3944 - accuracy: 0.8837 - val_loss: nan - val_accuracy: 0.9144
Epoch 4/10
1724/1724 [==============================] - 37s 22ms/step - loss: 0.2378 - accuracy: 0.9322 - val_loss: nan - val_accuracy: 0.9460
Epoch 5/10
1724/1724 [==============================] - 38s 22ms/step - loss: 0.1661 - accuracy: 0.9514 - val_loss: nan - val_accuracy: 0.9563
Epoch 6/10
1724/1724 [==============================] - 37s 22ms/step - loss: 0.1305 - accuracy: 0.9611 - val_loss: nan - val_accuracy: 0.9517
Epoch 7/10
1724/1724 [==============================] - 38s 22ms/step - loss: 0.1163 - accuracy: 0.9642 - val_loss: nan - v

## Prediction (IMPLEMENTATION)

In [22]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = model_final(x.shape, y.shape[1], len(x_tk.word_index)+1, len(y_tk.word_index)+1)
    model.fit(x, y, batch_size=BATCH_SIZE, epochs=EPOCH, validation_split=0.2)

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Epoch 1/10
1724/1724 [==============================] - 50s 24ms/step - loss: 1.2544 - accuracy: 0.6753 - val_loss: 0.7743 - val_accuracy: 0.7720
Epoch 2/10
1724/1724 [==============================] - 35s 20ms/step - loss: 0.6057 - accuracy: 0.8177 - val_loss: 0.5269 - val_accuracy: 0.8395
Epoch 3/10
1724/1724 [==============================] - 35s 20ms/step - loss: 0.3333 - accuracy: 0.9016 - val_loss: 0.2439 - val_accuracy: 0.9291
Epoch 4/10
1724/1724 [==============================] - 36s 21ms/step - loss: 0.1949 - accuracy: 0.9429 - val_loss: 0.1660 - val_accuracy: 0.9502
Epoch 5/10
1724/1724 [==============================] - 35s 21ms/step - loss: 0.1429 - accuracy: 0.9566 - val_loss: 0.1359 - val_accuracy: 0.9586
Epoch 6/10
1724/1724 [==============================] - 35s 21ms/step - loss: 0.1198 - accuracy: 0.9630 - val_loss: 0.1335 - val_accuracy: 0.9579
Epoch 7/10
1724/1724 [==============================] - 36s 21ms/step - loss: 0.1058 - accuracy: 0.9672 - val_loss: 0.1058 -

## Submission
When you're ready to submit, complete the following steps:
1. Review the rubric to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [18]:
# !pip install jupyter_contrib_nbextensions

In [17]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb --to html

['[NbConvertApp] Converting notebook machine_translation-zh.ipynb to html',
 '[NbConvertApp] Writing 336925 bytes to machine_translation-zh.html',
 '[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 395474 bytes to machine_translation.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?